In [1]:
#Data Pre-processing part
import os
import random
import pandas as pd

In [2]:
train_path = "/hdd/Gothi_raj/Whisper/dataset/kathbath/kb_data_clean_wav/gujarati/train/bucket.csv"
# dev_path = "/hdd/Gothi_raj/Whisper/dataset/kathbath/kb_data_clean_wav/gujarati/valid/bucket.csv"
text_save_path = '../dataset/kathbath/kb_data_clean_wav/gujarati/all_text.txt'
tokenizer_save_path = "../tokenizer/gujarati_tokenizer_1000"


audio = []
transcript= []

flg=0
with open(train_path,'r') as f:
    lines = f.readlines()
    for line in lines:
        # print(line)
        if flg==0:        
            flg=1
            continue
        line = line.split(',')
        audio.append(line[0])
        transcript.append(line[1])

In [3]:
train_df = pd.DataFrame({'path':audio,'transcription':transcript})

In [4]:
# # dev_path = "/raid/speech/rajgothi/whisper/kathbath/kb_data_clean_wav/gujarati/valid/bucket.csv"

# audio = []
# transcript= []

# flg=0
# with open(dev_path,'r') as f:
#     lines = f.readlines()
#     for line in lines:
#         # print(line)
#         if flg==0:        
#             flg=1
#             continue
#         line = line.split(',')
#         audio.append(line[0])
#         transcript.append(line[1])

In [5]:
# dev_df = pd.DataFrame({'path':audio,'transcription':transcript})

In [6]:
from datasets import DatasetDict,Dataset

train_data_hf = Dataset.from_pandas(train_df)
# dev_data_hf = Dataset.from_pandas(dev_df)

# dataset = DatasetDict({'train':train_data_hf,'validation':dev_data_hf})
dataset = DatasetDict({'train':train_data_hf})
dataset


/hdd/Gothi_raj/envs/wt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['path', 'transcription'],
        num_rows: 71063
    })
})

In [7]:
train_transcription = dataset['train']['transcription']
# validation_transcription = dataset['validation']['transcription']

In [8]:
train_transcription = set(train_transcription)

In [9]:
with open(text_save_path,'w') as f:
    for sentence in train_transcription:
        f.write(sentence+"\n")

In [10]:
# new tokenizer
from tokenizers import (decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer)
from transformers import GPT2Tokenizer, GPT2TokenizerFast, GPT2Model, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import WhisperTokenizer


new_tokenizer = Tokenizer(models.BPE())
new_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
trainer = trainers.BpeTrainer(vocab_size=1000, special_tokens=["<|endoftext|>"])
train_file = text_save_path
new_tokenizer.train([train_file], trainer=trainer)
new_tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
new_tokenizer.decoder = decoders.ByteLevel()

new_tokenizer = GPT2TokenizerFast(tokenizer_object=new_tokenizer)
new_tokenizer.save_pretrained("new_tokenizer_gpt2")
new_tokenizer

GPT2TokenizerFast(name_or_path='', vocab_size=1000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [11]:
# gpt2 tokenizer
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
print(len(gpt2_tokenizer.get_vocab()))
gpt2_tokenizer

# merge the vocabulary for the extended tokenizer
vocab_tokens = list(new_tokenizer.get_vocab())
decoded_tokens = [new_tokenizer.decoder.decode([token]) for token in vocab_tokens]
print(len(vocab_tokens), len(decoded_tokens))
gpt2_tokenizer.add_tokens(decoded_tokens)

gpt2_tokenizer.save_pretrained(tokenizer_save_path)
print(len(gpt2_tokenizer.get_vocab()))
gpt2_tokenizer

# validate the changes
text = "આ દવાઓ સર્જરી દરમિયાન જ્ઞ"
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
new_tokenizer = GPT2TokenizerFast.from_pretrained("new_tokenizer_gpt2")
extended_tokenizer = GPT2TokenizerFast.from_pretrained(tokenizer_save_path)

print(len(gpt2_tokenizer.encode(text)))
print(gpt2_tokenizer.encode(text))
print(len(new_tokenizer.encode(text)))
print(new_tokenizer.encode(text))
print(len(extended_tokenizer.encode(text)))
print(extended_tokenizer.encode(text))

50257
1000 1000
51199


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


67
[156, 103, 228, 220, 156, 103, 99, 156, 103, 113, 156, 103, 122, 156, 103, 241, 220, 156, 103, 116, 156, 103, 108, 156, 104, 235, 156, 103, 250, 156, 103, 108, 156, 104, 222, 220, 156, 103, 99, 156, 103, 108, 156, 103, 106, 156, 103, 123, 156, 103, 107, 156, 103, 122, 156, 103, 101, 220, 156, 103, 250, 156, 104, 235, 156, 103, 252]
16
[124, 555, 62, 123, 215, 65, 195, 67, 280, 75, 74, 62, 66, 98, 65, 410]
16
[50429, 50477, 50762, 50265, 51013, 51086, 50687, 50873, 50724, 50580, 50706, 50762, 50452, 51000, 51086, 50673]


In [12]:
decoded_tokens

[' પરન',
 'નજ',
 'યદ',
 ' મહત',
 'દલ',
 'ઉત',
 ' લગભગ',
 ' પડશ',
 'ઓ',
 ' નહ',
 'મન',
 ' રહ',
 'સક',
 'લવ',
 ' આપવ',
 ' અડ',
 'ચલ',
 ' પચ',
 'દન',
 'ખવ',
 'હન',
 'ઓએ',
 'ડક',
 'લક',
 'એન',
 'ળવવ',
 ' બત',
 'હમ',
 'વધ',
 '�',
 'ૈ',
 ' ઓર',
 ' તળ',
 ' એજન',
 ' આઈ',
 'ઓથ',
 'ટન',
 ' વસ',
 ' તપ',
 ' તસવ',
 'મસ',
 ' ભર',
 'પથ',
 'ડ',
 ' ઉઠ',
 'યમ',
 ' રચન',
 'ૂં',
 ' આઇ',
 ' દરવ',
 'એટલ',
 'બળ',
 'કથ',
 ' ઑ',
 'કવણ',
 ' ભય',
 ' એર',
 ' ઉજવણ',
 'તદ',
 'પવ',
 'ગસ',
 'ધર',
 'ગડ',
 'ષત',
 ' વર',
 'લર',
 'ટરમ',
 ' અમદ',
 ' જળ',
 'મણ',
 ' જગ',
 ' ઓફ',
 ' ઘર',
 'પજ',
 ' હ',
 ' અખ',
 ' સન',
 ' શર',
 'ઇન',
 'અમ',
 '�',
 ' બસ',
 ' રશ',
 'ભળ',
 'ઃ',
 'શનલ',
 'ળન',
 'પ',
 ' હક',
 'બઈ',
 ' ભક',
 'સઠ',
 'દરન',
 'યરન',
 ' અસરક',
 ' ઓ',
 'ૂ',
 ' ખસ',
 ' આશ',
 ' ઇ',
 'જપન',
 ' પડ',
 'ળમ',
 ' મથક',
 'નથ',
 ' નબળ',
 'રણ',
 ' કપ',
 '�',
 ' શ',
 ' એન',
 'ટલમ',
 'લગ',
 ' ઇમ',
 'તરપ',
 'ઈવ',
 '�',
 ' થય',
 ' આપણ',
 ' કરત',
 ' દ',
 ' બર',
 'શનમ',
 'નપ',
 '�',
 'ઈ',
 'જવ',
 ' થશ',
 'ુઃ',
 'ગત',
 'રલ',
 '�',
 ' ઉપલબ

In [117]:
# barakhadi =  '''
# ા	િ	ી	ુ	ૂ	ે	ૈ	ો	ૌ	ં	ઃ
# ક	કા	કિ	કી	કુ	કૂ	કે	કૈ	કો	કૌ	કં	કઃ
# ખ	ખા	ખિ	ખી	ખુ	ખૂ	ખે	ખૈ	ખો	ખૌ	ખં	ખઃ
# ગ	ગા	ગિ	ગી	ગુ	ગૂ	ગે	ગૈ	ગો	ગૌ	ગં	ગઃ
# ઘ	ઘા	ઘિ	ઘી	ઘુ	ઘૂ	ઘે	ઘૈ	ઘો	ઘૌ	ઘં	ઘઃ
# ચ	ચા	ચિ	ચી	ચુ	ચૂ	ચે	ચૈ	ચો	ચૌ	ચં	ચઃ
# છ	છા	છિ	છી	છુ	છૂ	છે	છૈ	છો	છૌ	છં	છઃ
# જ	જા	જિ	જી	જુ	જૂ	જે	જૈ	જો	જૌ	જં	જઃ
# ઝ	ઝા	ઝિ	ઝી	ઝુ	ઝૂ	ઝે	ઝૈ	ઝો	ઝૌ	ઝં	ઝઃ
# ટ	ટા	ટિ	ટી	ટુ	ટૂ	ટે	ટૈ	ટો	ટૌ	ટં	ટઃ
# ઠ	ઠા	ઠિ	ઠી	ઠુ	ઠૂ	ઠે	ઠૈ	ઠો	ઠૌ	ઠં	ઠઃ
# ડ	ડા	ડિ	ડી	ડુ	ડૂ	ડે	ડૈ	ડો	ડૌ	ડં	ડઃ
# ઢ	ઢા	ઢિ	ઢી	ઢુ	ઢૂ	ઢે	ઢૈ	ઢો	ઢૌ	ઢં	ઢઃ
# ણ	ણા	ણિ	ણી	ણુ	ણૂ	ણે	ણૈ	ણો	ણૌ	ણં	ણઃ
# ત	તા	તિ	તી	તુ	તૂ	તે	તૈ	તો	તૌ	તં	તઃ
# થ	થા	થિ	થી	થુ	થૂ	થે	થૈ	થો	થૌ	થં	થઃ
# દ	દા	દિ	દી	દુ	દૂ	દે	દૈ	દો	દૌ	દં	દઃ
# ધ	ધા	ધિ	ધી	ધુ	ધૂ	ધે	ધૈ	ધો	ધૌ	ધં	ધઃ
# ન	ના	નિ	ની	નુ	નૂ	ને	નૈ	નો	નૌ	નં	નઃ
# પ	પા	પિ	પી	પુ	પૂ	પે	પૈ	પો	પૌ	પં	પઃ
# ફ	ફા	ફિ	ફી	ફુ	ફૂ	ફે	ફૈ	ફો	ફૌ	ફં	ફઃ
# બ	બા	બિ	બી	બુ	બૂ	બે	બૈ	બો	બૌ	બં	બઃ
# ભ	ભા	ભિ	ભી	ભુ	ભૂ	ભે	ભૈ	ભો	ભૌ	ભં	ભઃ
# મ	મા	મિ	મી	મુ	મૂ	મે	મૈ	મો	મૌ	મં	મઃ
# ય	યા	યિ	યી	યુ	યૂ	યે	યૈ	યો	યૌ	યં	યઃ
# ર	રા	રિ	રી	રુ	રૂ	રે	રૈ	રો	રૌ	રં	રઃ
# લ	લા	લિ	લી	લુ	લૂ	લે	લૈ	લો	લૌ	લં	લઃ
# વ	વા	વિ	વી	વુ	વૂ	વે	વૈ	વો	વૌ	વં	વઃ
# શ	શા	શિ	શી	શુ	શૂ	શે	શૈ	શો	શૌ	શં	શઃ
# ષ	ષા	ષિ	ષી	ષુ	ષૂ	ષે	ષૈ	ષો	ષૌ	ષં	ષઃ
# સ	સા	સિ	સી	સુ	સૂ	સે	સૈ	સો	સૌ	સં	સઃ
# હ	હા	હિ	હી	હુ	હૂ	હે	હૈ	હો	હૌ	હં	હઃ
# ળ	ળા	ળિ	ળી	ળુ	ળૂ	ળે	ળૈ	ળો	ળૌ	ળં	ળઃ
# ક્ષ	ક્ષા	ક્ષિ	ક્ષી	ક્ષુ	ક્ષૂ	ક્ષે	ક્ષૈ	ક્ષો	ક્ષૌ	ક્ષં	ક્ષઃ
# જ્ઞ	જ્ઞા	જ્ઞિ	જ્ઞી	જ્ઞુ	જ્ઞૂ	જ્ઞે	જ્ઞૈ	જ્ઞો	જ્ઞૌ	જ્ઞં	જ્ઞઃ
# અ	
# આ	
# ઇ	
# ઈ	
# ઉ
# ઊ
# ઋ
# એ
# ઐ	
# ઓ	
# ઔ	
# અં	
# અ:
# '''

In [118]:
# barakhadi = barakhadi.split()

In [119]:
# barakhadi_new = [f' {i}' for i in barakhadi]

In [120]:
# decoded_tokens = barakhadi + barakhadi_new + decoded_tokens

In [121]:
# gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# print(len(gpt2_tokenizer.get_vocab()))
# gpt2_tokenizer
# gpt2_tokenizer.add_tokens(decoded_tokens)
# gpt2_tokenizer.save_pretrained("extended_tokenizer_gpt2")

50257


('extended_tokenizer_gpt2/tokenizer_config.json',
 'extended_tokenizer_gpt2/special_tokens_map.json',
 'extended_tokenizer_gpt2/vocab.json',
 'extended_tokenizer_gpt2/merges.txt',
 'extended_tokenizer_gpt2/added_tokens.json',
 'extended_tokenizer_gpt2/tokenizer.json')

In [61]:
# # gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# # print(len(gpt2_tokenizer.get_vocab()))
# # gpt2_tokenizer

# # merge the vocabulary for the extended tokenizer
# print(len(vocab_tokens), len(barakhadi))
# gpt2_tokenizer.add_tokens(barakhadi)
# gpt2_tokenizer.save_pretrained("extended_tokenizer_gpt2")

50257
1000 432


('extended_tokenizer_gpt2/tokenizer_config.json',
 'extended_tokenizer_gpt2/special_tokens_map.json',
 'extended_tokenizer_gpt2/vocab.json',
 'extended_tokenizer_gpt2/merges.txt',
 'extended_tokenizer_gpt2/added_tokens.json',
 'extended_tokenizer_gpt2/tokenizer.json')

In [137]:
# # text = "આ દવાઓ સર્જરી દરમિયાન"

# text = "આ દવાઓ સર્જરી દરમિયાન જ્ઞ"
# gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# extended_tokenizer = GPT2TokenizerFast.from_pretrained("extended_tokenizer_gpt2")

# print(len(gpt2_tokenizer.encode(text)))
# print(gpt2_tokenizer.encode(text))
# print(len(extended_tokenizer.encode(text)))
# print(extended_tokenizer.encode(text))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


67
[156, 103, 228, 220, 156, 103, 99, 156, 103, 113, 156, 103, 122, 156, 103, 241, 220, 156, 103, 116, 156, 103, 108, 156, 104, 235, 156, 103, 250, 156, 103, 108, 156, 104, 222, 220, 156, 103, 99, 156, 103, 108, 156, 103, 106, 156, 103, 123, 156, 103, 107, 156, 103, 122, 156, 103, 101, 220, 156, 103, 250, 156, 104, 235, 156, 103, 252]
13
[50677, 51521, 50257, 50685, 51399, 51800, 51781, 50259, 51670, 50258, 50545, 50472, 51096]


In [138]:
# extended_tokenizer.decode([50677, 51521, 50257, 50685, 51399, 51800, 51781, 50259, 51670, 50258, 50545, 50472])

# extended_tokenizer.decode([51521,50257,50685,51399,51800,51781,50259,51670,50258,50545,50472,51096])

' દવાઓ સર્જરી દરમિયાન જ્ઞ'

In [13]:
from transformers import WhisperFeatureExtractor

model_path = "openai/whisper-medium"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)

In [14]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_path, language="Gujarati", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
tokenizer.add_tokens(decoded_tokens)

942

In [16]:
print(len(tokenizer.get_vocab()))

52807


In [17]:
tokenizer.save_pretrained(tokenizer_save_path)

('../tokenizer/gujarati_tokenizer_1000/tokenizer_config.json',
 '../tokenizer/gujarati_tokenizer_1000/special_tokens_map.json',
 '../tokenizer/gujarati_tokenizer_1000/vocab.json',
 '../tokenizer/gujarati_tokenizer_1000/merges.txt',
 '../tokenizer/gujarati_tokenizer_1000/normalizer.json',
 '../tokenizer/gujarati_tokenizer_1000/added_tokens.json')

In [18]:
input_str = dataset["train"][0]["transcription"]

# prompt = "IndoAryan"
# prompt_ids = tokenizer.get_prompt_ids(prompt)

labels = tokenizer(input_str).input_ids

print('tokenized sentence length',len(labels))
# tokenizer = WhisperTokenizer.from_pretrained(model_path, language="Bengali", task="transcribe")
# tokenizer.add_tokens(decoded_tokens)
# labels = [51, 71, 72, 82, 53552, 82, 53869, 340, 2455, 83, 50257]

decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

tokenized sentence length 51
Input:                 વ્યવસાયોના બહુવિધ રેખાઓ ધરાવતા વ્યવસાયો માટે આવકને ફાળવવામાં આવવી જોઈએ
Decoded w/ special:    <|startoftranscript|><|gu|><|transcribe|><|notimestamps|>વ્યવસાયોના બહુવિધ રેખાઓ ધરાવતા વ્યવસાયો માટે આવકને ફાળવવામાં આવવી જોઈએ<|endoftext|>
Decoded w/out special: વ્યવસાયોના બહુવિધ રેખાઓ ધરાવતા વ્યવસાયો માટે આવકને ફાળવવામાં આવવી જોઈએ
Are equal:             True


In [45]:
dataset['train'][0]

{'path': '/hdd/Gothi_raj/Whisper/dataset/kathbath/kb_data_clean_wav/gujarati/train/audio/107/844424931458674-107-f.wav',
 'transcription': 'વ્યવસાયોના બહુવિધ રેખાઓ ધરાવતા વ્યવસાયો માટે આવકને ફાળવવામાં આવવી જોઈએ'}

In [98]:
# train_path = dataset['train']['path']
# new_path = []
# for path in train_path:
#     path = path.split('/')
#     path = '/'.join(path[:-1]) + '/train/' + path[-1]
#     new_path.append(path)

# dataset['train'] = dataset['train'].remove_columns(['path'])
# dataset['train'] = dataset['train'].add_column("path", new_path)

# dev_path = dataset['validation']['path']
# new_path = []
# for path in dev_path:
#     path = path.split('/')
#     path = '/'.join(path[:-1]) + '/dev/' + path[-1]
#     new_path.append(path)

# dataset['validation'] = dataset['validation'].remove_columns(['path'])
# dataset['validation'] = dataset['validation'].add_column("path", new_path)


# test_path = dataset['test']['path']
# new_path = []
# for path in test_path:
#     path = path.split('/')
#     path = '/'.join(path[:-1]) + '/test/' + path[-1]
#     new_path.append(path)

# dataset['test'] = dataset['test'].remove_columns(['path'])
# dataset['test'] = dataset['test'].add_column("path", new_path)

In [157]:
from transformers import WhisperProcessor

# processor = WhisperProcessor.from_pretrained(model_path, language="Hindi", task="transcribe")

processor = WhisperProcessor(tokenizer=tokenizer,feature_extractor=feature_extractor)
processor.save_pretrained("Trained_model/kathbath_token_processor_gu_1000")

[]

In [158]:
from datasets import Audio

dataset = dataset.cast_column("path", Audio(sampling_rate=16000))
dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'transcription'],
        num_rows: 71063
    })
    validation: Dataset({
        features: ['path', 'transcription'],
        num_rows: 3139
    })
})

In [159]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["path"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

In [160]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=128)

Map (num_proc=128):   0%|          | 0/71063 [00:00<?, ? examples/s]

Map (num_proc=128):   0%|          | 0/3139 [00:00<?, ? examples/s]

In [161]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 71063
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3139
    })
})

In [162]:
dataset.save_to_disk('Dataset/gujarati_kathbath_tokenized_1000')

Saving the dataset (0/137 shards):   0%|          | 0/71063 [00:00<?, ? examples/s]

Saving the dataset (0/7 shards):   0%|          | 0/3139 [00:00<?, ? examples/s]

In [80]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2120
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 239
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 418
    })
})